## Notebook 3: Análisis y Visualización
En este notebook realizaremos las siguientes operaciones:
1. Estadísticas: calcular estadísticas descriptivas como media, mediana, desviación estándar y percentiles.
2. Valores atípicos: detectar puntos de datos que son significativamente diferentes de otros.
3. Visualización:
    - Distribución de los datos: utilizar elementos como histogramas o boxplots para ver y entender mejor cómo se distribuyen los datos.
    - Correlaciones: podremos también utilizar gráficos de dispersión para identificar correlaciones.
4. Sacar conclusiones.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../")
from src import soporte_limpieza as sl
from src import soporte_variables as sv

pd.set_option('display.max_columns', None) 
# pd.options.display.float_format = '{:.2f}'.format #suprimir notación científica
pd.options.display.float_format = '{:,}'.format #usar separadores miles

In [20]:
# Importo un único dataframe que se ha unido en el notebook "Exploración", que contiene la información de los 9 años de análisis.

df = pd.read_csv("../datos/brasil_limpio.csv")

In [21]:
#Comprobamos integridad del DF
df.shape

(903723, 11)

In [22]:
df.columns

Index(['organo_superior', 'organo', 'unidad_gestora', 'categoria_economica',
       'origen_ingreso', 'tipo_ingreso', 'valor_previsto_actualizado',
       'valor_registrado', 'valor_ejecutado', 'porcentaje_ejecutado',
       'ano_ejercicio'],
      dtype='object')

In [23]:
df.dtypes

organo_superior                object
organo                         object
unidad_gestora                 object
categoria_economica            object
origen_ingreso                 object
tipo_ingreso                   object
valor_previsto_actualizado    float64
valor_registrado              float64
valor_ejecutado               float64
porcentaje_ejecutado          float64
ano_ejercicio                   int64
dtype: object

In [24]:
# E imprimimos una muestra para comprobar que los registros son los esperados.
df.sample(5)

,organo_superior,organo,unidad_gestora,categoria_economica,origen_ingreso,tipo_ingreso,valor_previsto_actualizado,valor_registrado,valor_ejecutado,porcentaje_ejecutado,ano_ejercicio
407871,Ministério da Educação,Universidade Federal de Lavras,UNIVERSIDADE FEDERAL DE LAVRAS,Ingresos Corrientes,Ingresos por Servicios,Servicios Administrativos y Comerciales Generales,0.0,0.0,"6,750.0",0.0,2018
93272,Ministério da Educação,Instituto Federal do Piauí,"INST.FED.DE EDUC.,CIENC.E TEC.DO PIAUI",Ingresos Corrientes,Ingresos por Servicios,Servicios Administrativos y Comerciales Generales,0.0,0.0,397.0,0.0,2016
18083,"Ministério da Agricultura, Pecuária e Abastec",Empresa Brasileira de Pesquisa Agropecuária,EMBRAPA-SETORIAL,Ingresos Corrientes,Ingresos por Servicios,Servicios Administrativos y Comerciales Generales,0.0,0.0,"9,896.5",0.0,2016
620125,Ministério da Infraestrutura,Agência Nacional de Aviação Civil,AGENCIA NACIONAL DE AVIACAO CIVIL,Ingresos Corrientes,Outras Receitas Correntes,"Multas administrativas, contractuales y judici...",0.0,0.0,"29,412.46",0.0,2019
461340,Ministério da Educação,Instituto Federal Catarinense,INSTITUTO FEDERAL CATARINENSE - REITORIA,Ingresos Corrientes,Ingresos por Servicios,Servicios Administrativos y Comerciales Generales,0.0,0.0,260.0,0.0,2018


In [25]:
# DE MOMENTO, HE ELIMINADO "FECHA_REGISTRO" PARA TRABAJAR DIRECTAMENTE CON LOS "ANO_EJERCICIO"
# Al exportar, tenemos nuevamente un problema con el tipo de dato de la fecha, que tendría que ser datetime, así que la pasaremos nuevamente por nuestra función.
# De momento no eliminaremos esta columa, aunque el "ano_ejercicio" podría ser suficiente si no requerimos, por ejemplo, de un análisis mensual.
# sl.conversion_fecha(df, "fecha_registro")

In [47]:
df.sample(1)[["valor_previsto_actualizado","valor_registrado","valor_ejecutado","porcentaje_ejecutado"]]

,valor_previsto_actualizado,valor_registrado,valor_ejecutado,porcentaje_ejecutado
260607,0.0,0.0,10.27,0.0
687392,0.0,0.0,"207,292.69",0.0
339712,0.0,0.0,"1,351.71",0.0
403746,0.0,0.0,"8,410,893.39",0.0
104798,0.0,0.0,187.44,0.0


In [27]:
# tendencia anual (análisis temporal)
# análisis por categoría económica y por órgano
# mencionar moneda
# outliers

Para empezar el análisis, conviene recordar los conceptos de partidas que tenemos, de acuerdo con el gobierno de Brasil:

- `valor previsto actualizado`: Monto actualizado del ingreso previsto.

- `valor registrado`: Monto que fue registrado como recaudado (ejecutado).

- `valor ejecutado`: Monto realmente recaudado.

- `porcentaje ejecutado`: Porcentaje de ejecución respecto al valor previsto.

In [58]:
# Calculamos la suma de los ingresos anuales previstos, registrados y ejecutados.

df_ingresos_anuales = round(df.groupby("ano_ejercicio")[["valor_previsto_actualizado","valor_registrado","valor_ejecutado"]].sum().reset_index(),0)
df_ingresos_anuales

,ano_ejercicio,valor_previsto_actualizado,valor_registrado,valor_ejecutado
0,2013,"1,878,907,730,497.0","36,097,705.0","1,577,778,876,823.0"
1,2014,"2,168,909,220,148.0","38,389,157.0","2,059,270,306,608.0"
2,2015,"2,556,218,693,479.0","60,645,454.0","2,396,627,354,081.0"
3,2016,"2,477,383,404,147.0","5,556,089.0","2,430,189,528,319.0"
4,2017,"3,302,128,310,472.0","1,585,463.0","2,183,460,599,708.0"
5,2018,"3,418,634,526,269.0","38,380,314.0","2,653,709,434,217.0"
6,2019,"2,886,115,355,751.0","219,911,638.0","2,709,765,467,311.0"
7,2020,"3,348,685,411,208.0","2,755,316,190.0","3,290,353,859,538.0"
8,2021,"3,856,437,321,456.0","14,621,809.0","3,494,440,511,726.0"


A primera vista, vemos que, hay diferencias significativas entre los montos previstos y los recaudados, pero principalmente en los que se registraron.

En cifras absolutas, es difícil interpretar estos datos, así que calcularemos ratios.

In [62]:
df_ingresos_anuales["ratio_previsto_recaudado"] = round(df_ingresos_anuales["valor_ejecutado"]/df_ingresos_anuales["valor_previsto_actualizado"] *100,0)

df_ingresos_anuales["ratio_registrado_recaudado"] = round(df_ingresos_anuales["valor_registrado"]/df_ingresos_anuales["valor_ejecutado"] *100,0)

df_ingresos_anuales

,ano_ejercicio,valor_previsto_actualizado,valor_registrado,valor_ejecutado,ratio_previsto_recaudado,ratio_registrado_recaudado
0,2013,"1,878,907,730,497.0","36,097,705.0","1,577,778,876,823.0",84.0,0.0
1,2014,"2,168,909,220,148.0","38,389,157.0","2,059,270,306,608.0",95.0,0.0
2,2015,"2,556,218,693,479.0","60,645,454.0","2,396,627,354,081.0",94.0,0.0
3,2016,"2,477,383,404,147.0","5,556,089.0","2,430,189,528,319.0",98.0,0.0
4,2017,"3,302,128,310,472.0","1,585,463.0","2,183,460,599,708.0",66.0,0.0
5,2018,"3,418,634,526,269.0","38,380,314.0","2,653,709,434,217.0",78.0,0.0
6,2019,"2,886,115,355,751.0","219,911,638.0","2,709,765,467,311.0",94.0,0.0
7,2020,"3,348,685,411,208.0","2,755,316,190.0","3,290,353,859,538.0",98.0,0.0
8,2021,"3,856,437,321,456.0","14,621,809.0","3,494,440,511,726.0",91.0,0.0


In [64]:
df_ingresos_anuales[["ratio_previsto_recaudado"]].mean()

ratio_previsto_recaudado   88.66666666666667
dtype: float64

In [65]:
df_ingresos_anuales[["ratio_registrado_recaudado"]].mean()

ratio_registrado_recaudado   0.0
dtype: float64

Ahora tenemos una vista más clara, y vemos que ocurren dos fenómenos:
1. De media, se recauda un 88.67% de las previsiones del gobierno.
2. Sin embargo, no se registra ni un 0% de lo que realmente se ingresa. Sin indagar más, se puede concluir que hay un descontrol absoluto sobre el registro de los ingresos, que implicaría la pérdida completa de trazabilidad de esos ingresos.

In [44]:
df.groupby("ano_ejercicio")[["valor_ejecutado"]].describe().reset_index()

ano_ejercicio valor_ejecutado                                             \
                          count                 mean                   std   
0          2013         3,843.0  410,559,166.4906142    7,643,673,625.6478   
1          2014         3,876.0  531,287,488.8049252  10,818,704,213.42943   
2          2015         3,935.0   609,053,965.458986 13,615,383,291.200209   
3          2016       172,831.0  14,061,074.27671396   880,914,190.3588994   
4          2017       164,821.0 13,247,466.037141018  437,682,197.43215287   
5          2018       152,820.0 17,364,935.441808533   639,570,998.5666689   
6          2019       154,762.0 17,509,243.013857927  493,464,437.85910374   
7          2020       127,001.0 25,908,094.105853498 1,149,685,853.3968027   
8          2021       119,834.0 29,160,676.533587545   716,570,707.4681927   

                                                                        \
                  min               25%       50%                  75%   
0     -111,172,781.62 585.9200000000001  50,000.0          959,437.185   
1     -624,211,482.24 516.8774999999999 46,652.08 1,034,421.1000000001   
2      -500,851,791.0             503.5 49,277.92   931,459.9199999999   
3 -156,285,934,188.03             210.0  2,139.11  28,186.754999999997   
4  -24,922,519,115.56             230.0  2,322.75            30,266.25   
5    -3,500,000,000.0            308.52  3,127.48          38,335.5875   
6  -60,439,234,089.12            348.62 3,432.545            40,846.42   
7   -6,300,814,890.95            376.42  4,765.85            61,575.38   
8  -12,126,486,130.06          356.2175 4,718.975  63,938.957500000004   

                      
                 max  
0 413,786,247,189.79  
1 548,491,208,660.31  
2 771,117,711,060.95  
3 199,853,249,683.03  
4 100,717,273,794.67  
5 169,256,367,162.37  
6  50,193,922,075.79  
7  350,022,251,304.3  
8  55,340,292,367.19

En este resumen, podemos ver que la cantidad de registros para los años 2013, 2014 y 2015 es muy inferior a la de años siguientes, sin embargo, la media de los ingresos no es tan distinta, sugiriendo serios errores a la hora de registrar los ingresos.

In [42]:
df.groupby("ano_ejercicio")[["valor_ejecutado"]].mean().median()

valor_ejecutado   25,908,094.105853498
dtype: float64

In [29]:
# Contamos la cantidad de registros por categoría económica
df.groupby("categoria_economica")[["valor_ejecutado"]].count()

,valor_ejecutado
categoria_economica,
Desconocido,16836
Ingresos Corrientes,846688
Ingresos Corrientes - intra-presupuestarios,14042
Ingresos de Capital,26074
Ingresos de Capital - intra-presupuestarios,83


In [30]:
# Los ingresos corrientes representan un 94% de los datos totales
round(df.groupby("categoria_economica")[["valor_ejecutado"]].count() / df.shape[0] * 100, 2)

,valor_ejecutado
categoria_economica,
Desconocido,1.86
Ingresos Corrientes,93.69
Ingresos Corrientes - intra-presupuestarios,1.55
Ingresos de Capital,2.89
Ingresos de Capital - intra-presupuestarios,0.01


In [31]:
# Montos totales en moneda
round(df.groupby("categoria_economica")[["valor_ejecutado"]].sum().sort_values(by = "valor_ejecutado"),0)

,valor_ejecutado
categoria_economica,
Ingresos de Capital - intra-presupuestarios,"20,096,501,113.0"
Ingresos Corrientes - intra-presupuestarios,"270,773,436,850.0"
Desconocido,"296,675,629,347.0"
Ingresos Corrientes,"10,947,368,785,703.0"
Ingresos de Capital,"11,260,681,585,317.0"


In [32]:
# Diferencia entre los montos previstos y los ejecutados

df_previsto_ejecutado = df[df["valor_ejecutado"].notna() & df["valor_previsto_actualizado"].notna()][["categoria_economica", "valor_ejecutado", "valor_previsto_actualizado"]]

df_previsto_ejecutado


,categoria_economica,valor_ejecutado,valor_previsto_actualizado
0,Ingresos Corrientes,"1,297.13",0.0
1,Ingresos Corrientes,"26,666,621.42",0.0
2,Ingresos Corrientes,"301,251.13",0.0
3,Ingresos Corrientes,"1,855.58",0.0
4,Ingresos Corrientes,"52,140.68",0.0
...,...,...,...
903718,Ingresos de Capital,0.0,"10,247,198.0"
903719,Ingresos de Capital,0.0,"16,940,891.0"
903720,Ingresos Corrientes,0.0,"200,000.0"
903721,Ingresos Corrientes,"1,000,000.0",0.0


In [33]:
# Para que los resultados sean interpretables, vamos a agrupar por categoría económica y ver los datos de manera porcentual.

df_previsto_ejecutado["diferencia"] = (df_previsto_ejecutado["valor_ejecutado"] - df_previsto_ejecutado["valor_previsto_actualizado"]) 

df_previsto_ejecutado

,categoria_economica,valor_ejecutado,valor_previsto_actualizado,diferencia
0,Ingresos Corrientes,"1,297.13",0.0,"1,297.13"
1,Ingresos Corrientes,"26,666,621.42",0.0,"26,666,621.42"
2,Ingresos Corrientes,"301,251.13",0.0,"301,251.13"
3,Ingresos Corrientes,"1,855.58",0.0,"1,855.58"
4,Ingresos Corrientes,"52,140.68",0.0,"52,140.68"
...,...,...,...,...
903718,Ingresos de Capital,0.0,"10,247,198.0","-10,247,198.0"
903719,Ingresos de Capital,0.0,"16,940,891.0","-16,940,891.0"
903720,Ingresos Corrientes,0.0,"200,000.0","-200,000.0"
903721,Ingresos Corrientes,"1,000,000.0",0.0,"1,000,000.0"


In [34]:
df_previsto_ejecutado["porcentaje"] = df_previsto_ejecutado["diferencia"] / df_previsto_ejecutado["valor_previsto_actualizado"] * 100
df_previsto_ejecutado["porcentaje"] = df_previsto_ejecutado["porcentaje"].replace({np.inf: 0})
df_previsto_ejecutado

,categoria_economica,valor_ejecutado,valor_previsto_actualizado,diferencia,porcentaje
0,Ingresos Corrientes,"1,297.13",0.0,"1,297.13",0.0
1,Ingresos Corrientes,"26,666,621.42",0.0,"26,666,621.42",0.0
2,Ingresos Corrientes,"301,251.13",0.0,"301,251.13",0.0
3,Ingresos Corrientes,"1,855.58",0.0,"1,855.58",0.0
4,Ingresos Corrientes,"52,140.68",0.0,"52,140.68",0.0
...,...,...,...,...,...
903718,Ingresos de Capital,0.0,"10,247,198.0","-10,247,198.0",-100.0
903719,Ingresos de Capital,0.0,"16,940,891.0","-16,940,891.0",-100.0
903720,Ingresos Corrientes,0.0,"200,000.0","-200,000.0",-100.0
903721,Ingresos Corrientes,"1,000,000.0",0.0,"1,000,000.0",0.0


In [35]:
round(df_previsto_ejecutado.groupby("categoria_economica")[["diferencia"]].mean(), 0)

,diferencia
categoria_economica,
Desconocido,"-42,442,894.0"
Ingresos Corrientes,"-574,579.0"
Ingresos Corrientes - intra-presupuestarios,"149,324.0"
Ingresos de Capital,"-73,182,146.0"
Ingresos de Capital - intra-presupuestarios,"111,904,694.0"


In [36]:
df_previsto_ejecutado.groupby("categoria_economica")["porcentaje"].agg(["mean", "median"])

,mean,median
categoria_economica,,
Desconocido,-inf,0.0
Ingresos Corrientes,-inf,0.0
Ingresos Corrientes - intra-presupuestarios,-inf,0.0
Ingresos de Capital,-inf,0.0
Ingresos de Capital - intra-presupuestarios,-inf,0.0


### Resumen y Conclusiones